# Target Name Extraction 

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests as req


In [116]:
#make target list by combing names from 3 databases to use as keywords 
target=[]
protein=pd.read_csv('pubChem_protein_list.csv')
gene=pd.read_csv('pubChem_gene_list.csv')
cell=pd.read_csv('pubChem_cell_list.csv')
for pro in protein['protacxn']:
    target.append(pro)
for ge in gene['genesymbol']:
    target.append(ge)
for cl in cell['name']:
    target.append(cl)

target.append('SARS-CoV-2') #additional addition outside of data
target = [*set(target)] # get rid of any duplicates 




C:\Users\Dan\AppData\Local\Temp\ipykernel_11964\2017711787.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  gene=pd.read_csv('pubChem_gene_list.csv')


In [122]:
#create list of synonyms of target from pubchem databases 
syn=list(protein['synos'])+list(gene['synos'])+list(cell['synos'])+list(protein['title'])
synonym=[]
for string in syn:
    if type(string)==str:
        new_list=string.rsplit('|') # split the lists of synonymns into seperate strings 
        for s in new_list:
            synonym.append(s)
synonym=[*set(synonym)] #remove any dupliactes 


In [115]:
#list of common words to ignore 
common_words=['for','FOR','For','cell','Cell','CELL','of','Of','OF','inhibitors','Inhibitors','INHIBITORS','entry','Entry','ENTRY','cytotoxicity','Cytotoxicity','CYTOTOXICITY','the','The','THE','this','This','THIS','true','True','TRUE','and','And','AND','with','With','WITH','panel','Panel','PANEL','in','In','IN','to','To','TO','anti','Anti','ANTI','species','Species','SPECIES','by','By','By','a','A','as','As','was','Was','WAS','viral','Viral','VIRAL','against','Against','AGAINST','or','Or','OR','all','All','ALL','at','At','AT','information','Information','INFORMATION','one','One','ONE','any','Any','ANY','end','End','END','body','BODY','Body','free','fast','best','base','pure','gram','gas','sex','term','axis','weak','AKA','aka','cow','hrs','HRS','Hrs','MAX','Max','max','AID','Aid','aid','min','Min','MIN','rat','ten','cat','NEW','New','new','eye','low','Low','LOW','hit','Not','NOT','not','twins','Activity','ACTIVITY','activity','bacterial','BACTERIAL','Bacterial','STRUCTURE','structure', 'Structure', 'Response','RESPONSE','response','TYPE','Type','type','Interaction','INTERACTION','interaction', 'Transport', 'Mutant', 'Single']

# Cleaning Target List

In [118]:
def isfloat(value):
    ''' isfloat checks if object can be transformed into a float
    value: any object to be checked 
    return: boolean 
    '''
    try:
        float(value)
        return True
    except ValueError:
        return False


In [117]:
target=[x for x in target if isfloat(x)==False] #remove numbers fom list 

for t in target:
    if t in common_words : #remove common words from list
        target.remove(t)


In [119]:
words_target=[] #list of separte words from target names 
for string in target:
    if type(string)==str:
        new_list=string.rsplit(' ') #split strings in target list into single words 
        for s in new_list:
            words_target.append(s) #add each word to new list 

words_target=[*set(words_target)] #remove any duplicate strings 

In [120]:
for t in words_target:
    if t in common_words :
        words_target.remove(t) #remove any common words from list

words_target=[x for x in words_target if isfloat(x)==False] #remove any strings that are numbers 

# Cleaning Synonym List

In [123]:
for s in synonym:
    if len(s)<3 or s in common_words :#remove any common words,2 letter words 
        synonym.remove(s)
synonym=[x for x in synonym if isfloat(x)==False] #remove any strings that are numbers 

In [125]:
words_syn=[]
for string in synonym:
    new_list=string.rsplit(' ')#split synonyms into separte strings fro each word
    for s in new_list:
        words_syn.append(s)
words_syn=[*set(words_syn)] #remove duplicates

In [126]:
for s in words_syn:
    if len(s)<3 or s in common_words : #remove common words,2 letter words 
        words_syn.remove(s) 
words_syn=[x for x in words_syn if isfloat(x)==False] #remove any strings that are numbers 

In [128]:
new_words_syn=[] #new edited list of synonyms split into words 
for w in words_syn:
    new_words_syn.append(w.replace(',',"").replace('(',"").replace(')',"").replace("\\"," "))#remove any punctuation marks
    

In [129]:
#secondary check to check for common words,small words or number strings 
for s in new_words_syn:
    if len(s)<3 or s in common_words :#remove common words or small 2 letter words 
        new_words_syn.remove(s)

new_words_syn=[x for x in new_words_syn if isfloat(x)==False] #remove any strings that are numbers   

In [130]:
#create new lists for each list with strings transformed to lowercase 
low_target=[]
for t in target:
    if type(t)==str and t not in common_words: #double check of filters before transforming 
        low_target.append(t.lower())
low_targetwords=[]
for t in words_target:
    if type(t)==str and t not in common_words:
        low_targetwords.append(t.lower())

low_syn=[]
for s in synonym:
    if s not in common_words :
        low_syn.append(s.lower())
    
low_syn_word=[]
for s in new_words_syn:
    if s not in common_words :
        low_syn_word.append(s.lower())

# Lists Created and Cleaned: 
target= list of target names 

low_target= list of target names in lowercase 

words_target= list of target names seperated into separte strings for each word

low_targetwords= list of target names in separte strings for each word in lowercase

synonym= list of synonyms of target names 

low_syn= list of synonyms of target names in lowercase 

new_words_syn= list of synonyms of target names in separte strings for each word

low_syn_word= list of synonyms of target names in separte strings for each word in lowercase

In [131]:
#get list of confirmatory AIDs
confirmatory=pd.read_csv('PubChem_Confirmatory_Bioassays .csv')
confirmatory_aid=list(confirmatory['aid']) #transform values of column into a list


In [137]:
#get list of Screening AIDs
screening=pd.read_csv('PubChem_screening_bioassay.csv')
screening_aid=list(screening['aid']) #transform values of column into a list


# XML parsing and Target Identifying

In [139]:
full_list_2=[] 

def assay_target_2(aid):
    """
    function that predicts target name of an assay, prints the answer and adds it to a list 
    aid: string PubChem unique identifier for bioassay
    
    return: none 
    """
    lis=[]
    string='https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/assay/'+aid+'/XML' #string of URL

    Web = req.get(string,verify=False) # Requesting for the website

    S = BeautifulSoup(Web.text,features="xml") # Creating a BeautifulSoup object and specifying the parser
    
    #first method: find single string with target name - found if file has specfic target section 
    for s in S.find_all('String'): #iterate through each tag found in file with the name string
        if s.text in target and s.text not in lis : #if it is in target list and not been found already then add it to list of potential targets 
            lis.append(s.text)
            
                
        if len(lis)==0: #if not in target list-try synonym list 
            if s.text in synonym and s.text not in lis :
                lis.append(s.text)
                
                
    
    if len(lis)==0: #if first method does not work: use 2nd method: finding target name from title
        
        title=S.find('RecordTitle').text #use tag name to find title section of file
        title_words=title.split() #get words from title 
        target_title = set(target)&set(title_words) # words that are in title and target list                     
        targetwords_title = set(words_target)&set(title_words) # words that are in title and target list 
        #for each of these sets of common strings, if they are not in list already then add
        for w in target_title:
            if w not in lis:
                lis.append(w)
                
        for w in targetwords_title:
            if w not in lis:
                lis.append(w)
               
    if len(lis)==0:  # if above showed no results, find any matches to synonym list 
        
        syn_title = set(new_words_syn)&set(title_words) #words that are in title and in synonym list
        
        for w in syn_title:#for the set of common strings, if they are not in list already then add
            if w not in lis:
                lis.append(w)   
                
    if len(lis)==0: # if above showed no results, change all lists to lowercase   
        
        low_title_words=[]                     
        for t in title_words:
            low_title_words.append(t.lower())
            
        
        low_target_title = set(low_target)&set(low_title_words) #words that are in title and target list 
        for w in low_target_title:#for the set of common strings, if they are not in list already then add
            if w not in lis:
                lis.append(w)   
               
        low_syn_title = set(low_syn_word)&set(low_title_words) #words that are in title and in synonym list
        for w in low_syn_title:#for the set of common strings, if they are not in list already then add
            if w not in lis:
                lis.append(w)
                
    if len(lis)==0: # if above showed no results, find all words used in xml file and find any matches to target list or synonym list
        dict_5={} 
        d=S.get_text() #list of all strings in text
        words=d.split() #list of strings split into individual words
        low_words=[]
        for w in words:
            low_words.append(w.lower()) #list of all words in lowercase
            
            
        #find common words between lists                     
        syn_words = set(new_words_syn)&set(words) #words that are in text and in synonym list
        target_words = set(target)&set(words) #words that are in text and target list 

        low_syn_words = set(low_syn_word)&set(low_words) #words that are in text and in synonym list
        low_target_words = set(low_targetwords)&set(low_words) #words that are in text and target list 
                     
        for t in syn_words: #find the number of times the word was found in the file 
            key=words.count(t)
            if key>2: #if it was more than 2 times, add word and number of times as key value pair in dictionary 
                dict_5[t] = key 
        #same as above for all other sets of common words 
        for t in target_words:
            key=words.count(t)
            if key>2:
                dict_5[t] = key
        
        if len(dict_5)==0: #if above didnt not show matches then use lowercase 
            
            for t in low_syn_words:
                key=words.count(t)
                if key>2:
                    dict_5[t] = key

            for t in low_target_words:
                key=words.count(t)
                if key>2:
                    dict_5[t] = key
    
        sorted_dict=sorted(dict_5.items(), key=lambda x:x[1],reverse=True) # sort matched words with most common last
        for key,value in sorted_dict:
            if key not in lis and len(lis)<5: #if word is not already in list and length of list is smaller than 5
                lis.append('???????') #add warning as this method is least accurate 
                lis.append(key) #add word to list
                
    full_list_2.append([S.RecordNumber.text,lis]) #add to final list with a list with first element the assay aid and then a list of the potential target names 
    print(S.RecordNumber.text) #print aid to show how much cod has been executed 

In [133]:
#find target names for each confirmatory assay in file 
for a in confirmatory_aid:
    assay_target_2(str(a)) 
    print(a)

C:\Users\Dan\New folder\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pubchem.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


411
411


C:\Users\Dan\New folder\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pubchem.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


519
519


C:\Users\Dan\New folder\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pubchem.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

1745852
1745852


C:\Users\Dan\New folder\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pubchem.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


1745854
1745854


C:\Users\Dan\New folder\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pubchem.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


1745855
1745855


In [140]:
#find target names for each screening assay in file 
for a in screening_aid:
    assay_target_2(str(a))
    print(a)#print aid to show function has completed 

C:\Users\Dan\New folder\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pubchem.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


362
362


C:\Users\Dan\New folder\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pubchem.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


365
365


C:\Users\Dan\New folder\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pubchem.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

1794799
1794799


C:\Users\Dan\New folder\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pubchem.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


1794800
1794800


C:\Users\Dan\New folder\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pubchem.ncbi.nlm.nih.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


1794835
1794835


In [136]:
# create new csv file with column 1 for AIDs and column to the potential targets for confimratory 
col_1=[]
col_2=[]
for a in full_list_2:
    col_1.append(a[0])
    col_2.append(a[1])
confirmatory_out_fin=pd.DataFrame(data={'AID':col_1,'Potential Target':col_2})
confirmatory_out_fin.to_csv('confirmatory_target_output.csv',sep=',',index=False)

In [141]:
# create new csv file with column 1 for AIDs and column to the potential targets for screening
col_1=[]
col_2=[]
for a in full_list_2:
    col_1.append(a[0])
    col_2.append(a[1])
screening_out_fin=pd.DataFrame(data={'AID':col_1,'Potential Target':col_2})
screening_out_fin.to_csv('screening_target_output.csv',sep=',',index=False)